In [1]:
from langchain_core.prompts import ChatPromptTemplate

In [2]:
template = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

In [3]:
template

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your name is {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing well, thanks!"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])

In [27]:
template2 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

In [28]:
template2.pretty_print()

================================ System Message ================================

You are a helpful AI bot. Your name is {name}.

================================ Human Message =================================

Hello, how are you doing?

================================== AI Message ==================================

I'm doing well, thanks!

================================ Human Message =================================

{user_input}


In [7]:
prompt_value = template.invoke(
    {"name": "Bob", "user_input": "What is your name?"}
)

In [8]:
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

In [14]:
template = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI bot."),
        # Means the template will receive an optional list of messages under
        # the "conversation" key
        ("placeholder", "{conversation}"),
        # Equivalently:
        # MessagesPlaceholder(variable_name="conversation", optional=True)
    ]
)

prompt_value = template.invoke(
    {
        "conversation": [
            ("human", "Hi!"),
            ("ai", "How can I assist you today?"),
            ("human", "Can you make me an ice cream sundae?"),
            ("ai", "No."),
        ]
    }
)

In [26]:
template.pretty_print()

================================ System Message ================================

You are a helpful AI bot.

============================= Messages Placeholder =============================

{conversation}


In [18]:
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot.', additional_kwargs={}, response_metadata={})])

In [17]:
prompt_value = template.invoke({})

In [19]:
from langchain_core.runnables import RunnableLambda


async def reverse(s: str) -> str:
    return s[::-1]


chain = RunnableLambda(func=reverse)

events = [
    event async for event in chain.astream_events("hello", version="v2")
]

In [20]:
events

[{'event': 'on_chain_start',
  'data': {'input': 'hello'},
  'name': 'reverse',
  'tags': [],
  'run_id': '8af127dd-9475-4019-aeb6-a18ea9a5a671',
  'metadata': {},
  'parent_ids': []},
 {'event': 'on_chain_stream',
  'run_id': '8af127dd-9475-4019-aeb6-a18ea9a5a671',
  'name': 'reverse',
  'tags': [],
  'metadata': {},
  'data': {'chunk': 'olleh'},
  'parent_ids': []},
 {'event': 'on_chain_end',
  'data': {'output': 'olleh'},
  'run_id': '8af127dd-9475-4019-aeb6-a18ea9a5a671',
  'name': 'reverse',
  'tags': [],
  'metadata': {},
  'parent_ids': []}]

In [24]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda, RunnableConfig
import asyncio


async def slow_thing(some_input: str, config: RunnableConfig) -> str:
    """Do something that takes a long time."""
    await asyncio.sleep(1) # Placeholder for some slow operation
    await adispatch_custom_event(
        "progress_event",
        {"message": "Finished step 1 of 3"},
        config=config # Must be included for python < 3.10
    )
    await asyncio.sleep(1) # Placeholder for some slow operation
    await adispatch_custom_event(
        "progress_event",
        {"message": "Finished step 2 of 3"},
        config=config # Must be included for python < 3.10
    )
    await asyncio.sleep(1) # Placeholder for some slow operation
    return "Done"

slow_thing1 = RunnableLambda(slow_thing)

async for event in slow_thing1.astream_events("some_input1", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'some_input1'}, 'name': 'slow_thing', 'tags': [], 'run_id': '650bde20-d344-43ea-b673-d63471c07b6b', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '650bde20-d344-43ea-b673-d63471c07b6b', 'name': 'progress_event', 'tags': [], 'metadata': {}, 'data': {'message': 'Finished step 1 of 3'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '650bde20-d344-43ea-b673-d63471c07b6b', 'name': 'progress_event', 'tags': [], 'metadata': {}, 'data': {'message': 'Finished step 2 of 3'}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': '650bde20-d344-43ea-b673-d63471c07b6b', 'name': 'slow_thing', 'tags': [], 'metadata': {}, 'data': {'chunk': 'Done'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'Done'}, 'run_id': '650bde20-d344-43ea-b673-d63471c07b6b', 'name': 'slow_thing', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [25]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="llama2")

# Without bind.
chain = llm | StrOutputParser()

chain.invoke("Repeat quoted words exactly: 'One two three four five.'")
# Output is 'One two three four five.'

# With bind.
chain = llm.bind(stop=["three"]) | StrOutputParser()

ModuleNotFoundError: No module named 'langchain_ollama'